In [ ]:
%qtconsole

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import math
import operator
import os
import json
import datetime
import time
from collections import defaultdict
from tqdm import tqdm

In [2]:
from my_twitter import *
from my_config import *

In [3]:
filename = "../data/10 - " + user_name + "_" + date + ".friends_enriched"
with open(filename, "r") as friends_file:
    friends = json.load(friends_file)

In [4]:
dirpath = "../data/11 - " + user_name + "_" + date + "_friends_tweets"

In [5]:
authors_tweetcount = defaultdict(int)
authors_uni_tc = defaultdict(int)
authors_rtcount = defaultdict(int)
authors_tweetlist = defaultdict(list)

In [6]:
edges = []

In [7]:
for author in tqdm(list(friends)):
    
    with open(dirpath + "/" + author["id_str"] + ".tweets","r") as file:
        tweets = json.load(file)
        for tweet in tweets:
            tweet = defaultdict(bool,tweet)
            for field in tweet.keys():
                if type(tweet[field]) is dict: tweet[field] = defaultdict(bool,tweet[field]
                                                                         )
            authors_tweetcount[tweet["user"]["id_str"]] += 1

            if "retweeted_status" not in tweet.keys():
                authors_uni_tc[ tweet["user"]["id_str"] ] += 1
#                 authors_tweetlist[tweet["user"]["id_str"]].append({
#                         "id_str" : tweet["id_str"],
#                         "created_at" : tweet["created_at"],
#                         "text" : tweet["text"],
#                         "times_retweeted" : tweet["retweet_count"]
#                     })
                
            else:
                authors_rtcount[tweet["user"]["id_str"]] += 1
                edges.append( 
                    {
                        "edge" : (tweet["user"]["id_str"],
                                  tweet["retweeted_status"]["user"]["id_str"]),
                        
                        "tweet" : {
                                    "id_str" : tweet["retweeted_status"]["id_str"],
                                    "created_at" : tweet["retweeted_status"]["created_at"],
                                    "text" : tweet["retweeted_status"]["text"]
                                },
                        
                        "rtdate" : tweet["created_at"]
                    }
                )

100%|██████████████████████████████████████████████████████████████████████████████| 3396/3396 [04:14<00:00,  3.90it/s]


In [8]:
weighted_edges = [
    (
        edge["edge"][0],
        edge["edge"][1],
        {
            "weight" : 1./authors_rtcount[edge["edge"][0]],
            "rtdate" : edge["rtdate"],
            "tweetid" : edge["tweet"]["id_str"],
            "tweetdate" : edge["tweet"]["created_at"],
            "tweettext" : edge["tweet"]["text"]
        }
    )
    for edge in edges
]

In [9]:
G_RT=nx.MultiDiGraph()
G_RT.add_edges_from(weighted_edges)
G_RT.add_nodes_from([(f["id_str"], f) for f in friends])

In [10]:
for element in G_RT.nodes():
    if element not in [f["id_str"] for f in friends]:
        G_RT.remove_node(element)

In [11]:
for node in G_RT.nodes():
    G_RT.node[node]["tweetcount"] = authors_tweetcount[node]
    G_RT.node[node]["uni_tc"] = authors_uni_tc[node]
    G_RT.node[node]["rtcount"] = authors_rtcount[node]
#    G_RT.node[node]["tweetlist"] = authors_tweetlist[node]

In [66]:
type(None)

NoneType

In [65]:
[(k, type(v),v) for k,v in G_RT.node["60033181"].items() if type(v) not in [bool,str, int, dict]]

[('pagerank_rt', float, 6.834353976328872e-05),
 ('muting', NoneType, None),
 ('follow_request_sent', NoneType, None),
 ('blocked_by', NoneType, None),
 ('notifications', NoneType, None),
 ('following', NoneType, None),
 ('blocking', NoneType, None)]

In [67]:
def graphml_supported(G):
    G_copy = G.copy()
    for node in G_copy.nodes():
        G_copy.node[node] = {k:v for k,v in G_copy.node[node].items()
                             if type(v) not in [dict, type(None)]}
    return(G_copy)

In [28]:
H = nx.DiGraph()
H.add_nodes_from(G_RT.nodes())
for u,v,d in G_RT.edges(data=True):
    w = d['weight']
    if H.has_edge(u,v):
        H[u][v]['weight'] += w
    else:
        H.add_edge(u,v,weight=w)

In [29]:
pr = nx.pagerank(H)

In [30]:
for node in G_RT.nodes():
    G_RT.node[node]["pagerank_rt"] = pr[node]
    G_RT.node[node]["indegree_rt"] = G_RT.in_degree(node)

In [ ]:
nx.write_graphml(graphml_supported(G_RT), "../output/21 - graph_retweets_" + user_name + "_" + date +".graphml", prettyprint=False)

In [32]:
nx.write_gpickle(G_RT, "../temp/21 - graph_retweets_" + user_name + "_" + date +".gpickle")

In [ ]:
%qtconsole